In [ ]:
import cv2
import numpy as np

    
cap0 = cv2.VideoCapture(0) #camera NO.1
cap1 = cv2.VideoCapture(1) #camera NO.2


def draw_direction(img, lx, ly, nx, ny): #Calculate the moving direction arrow based on the previous ball position and the current position
    dx = nx - lx
    dy = ny - ly
    if abs(dx) < 4 and abs(dy) < 4:
        dx = 0
        dy = 0
    else:
        r = (dx**2 + dy**2)**0.5
        dx = int(dx/r*40)
        dy = int(dy/r*40)
        # print(dx, dy)
    cv2.arrowedLine(img, (60, 100), (60+dx, 100+dy), (0, 255, 0), 2)
    
def draw_direction1(imgs, lx1, ly1, nx1, ny1): #Calculate the moving direction arrow based on the previous ball position and the current position
    dx1 = nx1 - lx1
    dy1 = ny1 - ly1
    if abs(dx1) < 4 and abs(dy1) < 4:
        dx1 = 0
        dy1 = 0
    else:
        r = (dx1**2 + dy1**2)**0.5
        dx1 = int(dx1/r*40)
        dy1 = int(dy1/r*40)

    cv2.arrowedLine(imgs, (60, 100), (60+dx1, 100+dy1), (0, 255, 0), 2)
    
############CAM1########################################################################
def imgfindcolor(capin):   #find colors
    lower = np.array([2, 100, 100])     # Suitable for orange table tennis 4<=h<=32  
    upper = np.array([27, 255, 255])                               
                                                     
    global targetPos_x
    targetPos_x = 0     # x-coordinate obtained by color detection
    global targetPos_y
    targetPos_y = 0     # y-coordinate obtained by color detection
    global lastPos_x
    lastPos_x = 0       # x-coordinate obtained by the color detection of the previous frame of image
    global lastPos_y
    lastPos_y = 0       # y-coordinate obtained by the color detection of the previous frame of image
    _, img = capin.read()
    b, g, r = cv2.split(img)    # Separate three colors
    r = np.int16(r)             # Convert red and blue to int16
    b = np.int16(b)
    r_minus_b = r - b           # Subtract the blue channel from the red channel to get "r_minus_b"
    r_minus_b = (r_minus_b + abs(r_minus_b)) / 2    # "r_minus_b" less than 0 are converted to 0
    r_minus_b = np.uint8(r_minus_b)                 # Convert the data type back to "uint8"
    
    imgHsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    imgMask = cv2.inRange(imgHsv, lower, upper)     # Mask
    imgOutput = cv2.bitwise_and(img, img, mask=imgMask)
    imgbyinput, contours, hierarchy = cv2.findContours(imgMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)   # Find contours
    
    imgMask = cv2.cvtColor(imgMask, cv2.COLOR_GRAY2BGR) 
    x, y, w, h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        if area > 300:
            x, y, w, h = cv2.boundingRect(cnt) # x, y are the coordinates of the upper left point of the matrix, w, h are the width and height of the matrix
            lastPos_x = targetPos_x
            lastPos_y = targetPos_y
            targetPos_x = int(x+w/2)
            targetPos_y = int(y+h/2)
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(img, (targetPos_x, targetPos_y), 2, (0, 255, 0), 4)
    # Coordinates (within the image)
    cv2.putText(img, "({:0<2d}, {:0<2d})".format(targetPos_x, targetPos_y), (20, 30),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)  # Text
    draw_direction(img, lastPos_x, lastPos_y, targetPos_x, targetPos_y)

    return img 

############CAM2#########################################################################
def imgfindcolor1(capin):    #find colors
    lower1 = np.array([2, 100, 100])     # Suitable for orange table tennis 4<=h<=32
    upper1 = np.array([27, 255, 255])
    global targetPos1_x
    targetPos1_x = 0     # x-coordinate obtained by color detection
    global targetPos1_y
    targetPos1_y = 0     # y-coordinate obtained by color detection
    global lastPos1_x
    lastPos1_x = 0       # x-coordinate obtained by the color detection of the previous frame of image
    global lastPos1_y
    lastPos1_y = 0       # y-coordinate obtained by the color detection of the previous frame of image
    _, imgs = capin.read()
    b, g, r = cv2.split(imgs)    # Separate three colors
    r = np.int16(r)             # Convert red and blue to int16
    b = np.int16(b)
    r_minus_b = r - b           # Subtract the blue channel from the red channel to get "r_minus_b"
    r_minus_b = (r_minus_b + abs(r_minus_b)) / 2    # "r_minus_b" less than 0 are converted to 0
    r_minus_b = np.uint8(r_minus_b)                 # Convert the data type back to "uint8"
    
    imgHsv1 = cv2.cvtColor(imgs, cv2.COLOR_BGR2HSV)
    imgMask1 = cv2.inRange(imgHsv1, lower1, upper1)     # Mask
    imgOutput1 = cv2.bitwise_and(imgs, imgs, mask=imgMask1)
    imgbyinput, contours, hierarchy = cv2.findContours(imgMask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)   # Find contours   
    
    imgMask1 = cv2.cvtColor(imgMask1, cv2.COLOR_GRAY2BGR) 
    x, y, w, h = 0, 0, 0, 0
    for cnt in contours:
        area = cv2.contourArea(cnt)

        if area > 300:
            x, y, w, h = cv2.boundingRect(cnt) # x, y are the coordinates of the upper left point of the matrix, w, h are the width and height of the matrix
            lastPos1_x = targetPos1_x
            lastPos1_y = targetPos1_y
            targetPos1_x = int(x+w/2)
            targetPos1_y = int(y+h/2)
            cv2.rectangle(imgs, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.circle(imgs, (targetPos1_x, targetPos1_y), 2, (0, 255, 0), 4)
    # Coordinates (within the image)
    cv2.putText(imgs, "({:0<2d}, {:0<2d})".format(targetPos1_x, targetPos1_y), (20, 30),
                cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 2)  # Text
    draw_direction1(imgs, lastPos1_x, lastPos1_y, targetPos1_x, targetPos1_y)
   
    return imgs

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 
imgfindcolor(cap0)
imgfindcolor1(cap1)

while True:
    # Output parameters
    img0 = imgfindcolor(cap0)
    img1 = imgfindcolor1(cap1)
    cv2.imshow('cap1', img0) # Screen display of cam1
    cv2.imshow('cap2', img1) # Screen display of cam2

#<<<<<<<<<<<<<<Calculate ball position coordinates<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<    
    A = np.array([                          #
            [ 0.4855,   0.8075,  -0.2054], 
            [-0.2805,   0.0139,  -0.8023],
            [-1.3411,  -0.7169,   0.1418],
            [ 0.1980,  -0.0016,  -0.6909]     
])

    Y = np.array([ targetPos_x - 168.6745, targetPos_x - 851.4135, targetPos1_x - 1716.9, targetPos1_y - 704.5 ])
    X = np.linalg.inv(A.T.dot(A)).dot(A.T.dot(Y))

    print(X)   # Calculate 2 cameras to get the 3D coordinates of ball
    
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<        
    if cv2.waitKey(1) & 0xFF == ord('q'):          # Press "q" to exit 
        print("Quit\n")
        break
cap0.release()
cap1.release()
cv2.destroyAllWindows()
